<a href="https://colab.research.google.com/github/elliewalters-oss/Data-Science/blob/main/Foundations_of_DS_08_Handling_Missing_Data_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(readr)
library(dplyr)
library(tidyr)
library(forcats)

In [ ]:
system("gdown 1tg9T97GfwstlY1R5AamcDkJuDFM01oL5")

# Missing data

In this lecture notes, we will deal with a dataset for a predictive task.
If such a dataset has $p+1$ columns, we assume that $p$ are the independent variables (also known as the features, or the inputs) and one column is the dependent variable (also known as the label or the output).
Eventually, our task will be that of devising and training a machine learning model that predicts the label based on the features.

Missing data are values at specific rows (i.e., observations) and columns (i.e., features) that are not available in the dataset.
It is important to remark that, to train a prediction model, missing data can be in one of the feature columns but **not** in the label column.
An observation without a valid label is not serviceable and must be discarded.

When a piece of data seems to be missing from a dataset, i.e., there is an "empty" cell in a tabular dataset, we must first determine if the missingness of the data is a valid state or not.
To make an example, if a column reports the environment temperature in a room, and one value is missing, this is not a valid state.
In real life, the room certainly had a temperature and not having recorded it is a "defect" in our data, but does not correspond to a valid world-state.
On the other hand, if a column reports the date when a customer ended their subscription, we would expect this column to have missing data for all the current customers (i.e., those who have not ended their subscription).

If we can determine that the missingness of the value is *not* a valid state, we can attempt to classify it into one of three common types: missing completely at random (MCAR), missing at random (MAR), and missing not at random (MNAR).

Values are **missing completely at random** if their missingness is unrelated to both observable and unobservable variables.
For example, consider a temperature sensor that sometimes stops working, and it is not possible to associate this malfunction with any other environmental parameter (both inside and outside of our dataset).
In this case, the temperature values that are not reported due to the sensor's malfunction are missing completely at random.
Remark that, for MCAR data, the probability that a value is missing is independent of the data and is the same for all values.

Values are **missing at random** if their missingness depends on some observed property of the data.
For example, if the temperature sensor is more likely to stop working in highly humid environments, then data recorded where humidity is high has a higher chance of containing a missing value.
If the humidity level is not recorded, there is no way to determine this correlation from the data.
But if the humidity level *is* recorded, a preliminary data analysis might reveal the correlation between humidity and the probability that the temperature is missing.
It's important to remark that, assuming that humidity is the only factor affecting the sensor's reliability, if we could separate our data by humidity, missing data *for a given humidity level* would be MCAR.

Values are **missing not at random** when the missingness of a value is related to the value that this variable would have if it were not missing.
Imagine that the temperature sensor has a high chance of stop working at high temperatures.
Then, high temperature values are more likely to be missing.
But, precisely because these values are missing, we have no direct way to suspect that high temperatures are associated with a higher chance of missing values.

# A practical example

In [ ]:
d <- read_csv('/content/titanic.csv')

Rows: 1309 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): pclass, survived, name, sex, ticket, cabin, embarkment_port, lifeboat
dbl (5): age, siblings_and_spouses_onboard, parents_and_children_onboard, fa...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In this example, we examine data about the passengers on board the Titanic.
The data has the following features:
* `pclass`: the class the passenger was travelling in (categorical: first, second, or third).
* `name`: the passenger's name (a string).
* `sex`: the passenger's sex (categorical: male or female).
* `age`: the passenger's age (non-negative integer).
* `siblings_and_spouses_onboard`: number of siblings and spouse(s) the passenger had onboard with them (non-negative integer).
* `parents_and_children_onboard`: number of parents and children the passenger had onboard with them (non-negative integer).
* `ticket`: passenger's ticket number, as it was sold by the navigation company (numerical string).
* `fare`: fare paid by the passenger.
* `cabin`: number/code of the cabin where the passenger was staying (string, it sometimes contains multiple space-separated values).
* `embarkment_port`: port where the passenger embarked (categorical: Cherbourg, Queenstown, Southampton).
* `lifeboat`: id code of the lifeboat the passenger got on, if any (string).
* `body_id_number`: id code of the passenger's body, if the passenger died and their body was retrieved (numerical string).

And the following label:
* `survived`: whether the passenger survived the shipwreck (categorical: Yes or No).

In [ ]:
# Mark categorical data as such
d <- d %>%
  mutate(
    pclass = as_factor(pclass),
    sex = as_factor(sex),
    survived = as_factor(survived),
    embarkment_port = as_factor(embarkment_port)
  )

In [ ]:
summary(d)

    pclass    survived      name               sex           age       
 First :323   Yes:500   Length:1309        female:466   Min.   : 0.17  
 Second:277   No :809   Class :character   male  :843   1st Qu.:21.00  
 Third :709             Mode  :character                Median :28.00  
                                                        Mean   :29.88  
                                                        3rd Qu.:39.00  
                                                        Max.   :80.00  
                                                        NA's   :263    
 siblings_and_spouses_onboard parents_and_children_onboard    ticket         
 Min.   :0.0000               Min.   :0.000                Length:1309       
 1st Qu.:0.0000               1st Qu.:0.000                Class :character  
 Median :0.0000               Median :0.000                Mode  :character  
 Mean   :0.4989               Mean   :0.385                                  
 3rd Qu.:1.0000               3rd 

Observations:

* Column `body_id_number` has many missing values, but not all of these are invalid. In fact, if a person survived, they will not have a body id number. And, even if they did not survive, their body might have never been recovered and, therefore, it is correct that they do not have a body id number.
* In any case, if we want to predict the label `survived` from the passengers' features, we should **remove** the column `body_id_number`. This column contains information that were only available after the shipwreck, so it should not be used to predict if a person could have survived the wreck. In particular, if an observation has a value in column `body_id_number`, then it surely died in the shipwreck.

In [ ]:
# Drop the body_id_number column
d <- d %>%
  select(!body_id_number)

More observations:
* Column `age` has missing values and these definitely do not correspond to valid states, i.e., every person has an age, no matter whether it is recorded in the data.
* Still, we don't know if this data is missing completely at random, at random, or not at random.
* For example, it is plausible to assume that onboard records of 3rd-class passengers were not as thorough as the records of 1st-class passengers.

In [ ]:
# Print the percentage of missing value in `val_col` in each group defined
# by `grp_col`.

pct_missing <- function(data, grp_col, val_col) {
  data %>%
    group_by(across(all_of(grp_col))) %>%
    summarise(pct_missing = mean(is.na(across(all_of(val_col)))) * 100)
}

Remarks about the above function:
*   We need to use `across()` because otherwise R expects to find a literal column name, e.g., in the `group_by()` function. I.e., if we did not use `across()`, and we just wrote `group_by(grp_col)`, R would be looking for a column called literaly "grp_col", rather than looking for the column whose name is contained in the `grp_col` string.
*   When using `across()`, we further have to use `all_of`, because across expects "tidyselect syntax" and not a raw string (see the documentation). Using `all_of` transforms our string into the appropriate object that is then passed to `across()`.

In [ ]:
# Percentage of missing values in column "age" in each group defined by column
# "pclass".
d %>%
  pct_missing(grp_col = "pclass", val_col = "age")

pclass,pct_missing
<fct>,<dbl>
First,12.074303
Second,5.776173
Third,29.337094


This theory might be plausible: third-class passengers have a much higher percentage of missing values.
Still, perhaps surprisngly, it's second-class passengers who have the fewest missing values, and not first-class ones.

Another hypothesis is that not many children got their age recorded.
If third-class passengers included predominantly emigrant families with many children, this might explain the missingness.

In [ ]:
# Average age per group defined by column "pclass".
d %>%
  group_by(pclass) %>%
  summarise(mean_age = mean(age, na.rm = TRUE))

pclass,mean_age
<fct>,<dbl>
First,39.15993
Second,29.50670
Third,24.81637


Indeed, the average (recorded) age among third-class passengers is the lowest.
Still, second-class passengers have a lower average age than first-class passengers, but a smaller fraction of missing data.

For the moment, it's hard to make further hypothesis about the missingness of the `age` feature.
Remark that our first hypothesis would have implied that values are missing at random (assuming that `pclass` was the *only* contributing factor to `age`'s missingness), while the second one would have implied that values are missing not at random.

More observations:
* The cabin and lifeboat columns have many missing values. However, in many instances, these missing values can correspond to valid world-states.
* For example, passengers in lower classes did not have cabins and were lodged in dormitories.
* And, unfortunately, not everyone had space on the lifeboats.

In [ ]:
# Percentage of missing values in column "cabin" in each group defined by column
# "pclass".
d %>%
  pct_missing(grp_col = "pclass", val_col = "cabin")

pclass,pct_missing
<fct>,<dbl>
First,20.74303
Second,91.69675
Third,97.74330


Indeed, almost no second- or third-class passneger has an assigned cabin.
Conversely, presumably all first-class passengers were assigned a cabin.
Still, more than 20% of data about first-class passengers has no information on their cabin and this is likely to be "really" missing data.

In [ ]:
# Percentage of missing values in column "lifeboat" in each group defined by\
# column "survived".
d %>%
  pct_missing(grp_col = "survived", val_col = "lifeboat")

survived,pct_missing
<fct>,<dbl>
Yes,4.60000
No,98.88752


As the above analysis shows, if we assume that having a missing value in column `lifeboat` means that the person did not make it into a lifeboat, then not making it is strongly correlated with not surviving.

In other words, if we assume that all the missing values in the `lifeboat` column correspond to the real-life scenario of a person not getting into a lifeboat, 98.9% of people who did not embark on a lifeboat died (and 1.1% survived), whereas only 4.6% of those who embarked a lifeboat died (and 95.6% survived).

More observations:
* Columns `fare` and `embarkment_port` have very few missing values. Is there something special about the observation with the missing values?

In [ ]:
# Rows with NA in the "fare" column.
d %>%
  filter(is.na(fare))

pclass,survived,name,sex,age,siblings_and_spouses_onboard,parents_and_children_onboard,ticket,fare,cabin,embarkment_port,lifeboat
<fct>,<fct>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<fct>,<chr>
Third,No,"Storey, Mr. Thomas",male,60.5,0,0,3701,NA,NA,Southampton,NA


A third-class passenger. Hard to believe that he was gifted a free ticket for being an influencer... this must be missing data!

In [ ]:
# Rows with NA in the "embarkment_port" column.
d %>%
  filter(is.na(embarkment_port))

pclass,survived,name,sex,age,siblings_and_spouses_onboard,parents_and_children_onboard,ticket,fare,cabin,embarkment_port,lifeboat
<fct>,<fct>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<fct>,<chr>
First,Yes,"Icard, Miss. Amelie",female,38,0,0,113572,80,B28,NA,6
First,Yes,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62,0,0,113572,80,B28,NA,6


# Dealing with missing data

Now that we have established that some data is missing in our dataset, we can develop strategies to deal with it.

Indeed, several machine learning algorithms cannot deal with missing data and require that there are no missing values in any of the feature columns (recall: in no case can there be missing values in the label column).

The first strategy to fill in missing data is to try to recover the correct value from alternative data sources.
For example, we can recover the embarkment port of Ms. Icard and Mrs. Stone from the [Encyclopedia Titanica](https://www.encyclopedia-titanica.org/).

In [ ]:
# After a quick search, the embarkment port for the two passengers with
# missing value is Southampton. Fill it in.
d <- d %>%
  mutate(embarkment_port = replace_na(embarkment_port, "Southampton"))

## Row or column removal

Another way of dealing with missing data is to remove either columns or row from the dataset.

* We can remove entire columns if they have a large share of missing values and very few valid values.
* We can remove entire rows if they have multiple missing values (i.e., they are low-quality observations) or even if they only have one missing value, but we have so much data that we can afford to reduce the number of observations.

Before evaluating if we should remove any column or row from our dataset, let us replace the values currently marked as missing but potentially corresponding to valid states.

In [ ]:
# Replace NA in the "lifeboat" column with the string "None", assuming
# that passengers with a missing lifeboat identifier did not make it
# into any lifeboat.
d <- d %>%
  mutate(lifeboat = replace_na(lifeboat, "None"))

We might have made some small error in the above process.
For example, we assume that all people without a valid `lifeboat` entry did not get into a lifeboat.
On the other hand, we cannot exclude the possibility that some missing values in the `lifeboat` column correspond to people who did get a spot on a lifeboat and this information is missing in our dataset.
Still, some compromise is necessary when dealing with a real-life dataset, and this is a reasonable one.

In [ ]:
summary(d)

    pclass    survived      name               sex           age       
 First :323   Yes:500   Length:1309        female:466   Min.   : 0.17  
 Second:277   No :809   Class :character   male  :843   1st Qu.:21.00  
 Third :709             Mode  :character                Median :28.00  
                                                        Mean   :29.88  
                                                        3rd Qu.:39.00  
                                                        Max.   :80.00  
                                                        NA's   :263    
 siblings_and_spouses_onboard parents_and_children_onboard    ticket         
 Min.   :0.0000               Min.   :0.000                Length:1309       
 1st Qu.:0.0000               1st Qu.:0.000                Class :character  
 Median :0.0000               Median :0.000                Mode  :character  
 Mean   :0.4989               Mean   :0.385                                  
 3rd Qu.:1.0000               3rd 

The `cabin` column has a very high percentage of missing value.
In this example, we remove it.

In [ ]:
# Remove the "cabin" column.
d <- d %>%
  select(!cabin)

In [ ]:
# How many rows contain >1 missing values?
# To do so, we only keep rows whose row-wise sum of is.na is larger than one.
d %>%
  filter(rowSums(is.na(.)) > 1) %>%
  nrow()

[1] 0

There is no observation with more than one missing value.
Let us not remove any row, then, and move to the next strategy: replacing missing values through imputation.

## Imputation

To impute a missing value means to replace it with some valid value.
There are many strategies for imputing data.

The most basic ones include replacing the missing value with a representative value from the dataset.
For numeric columns, this is usually the mean or the median.
For categorical columns, it is usually the mode.
In the most basic version of this imputation technique, these statistics are computed over (the valid entries of) the entire dataset.

A slightly more advanced technique computes the above statistics (mean, median or mode) on a subset of the data that shares some characteristics with the observation that we are imputing.
For example, we can impute the `fare` column with the mean fare for passengers in the same class.
Remark that we can use this method only for observations that do not have missing values in the `fare` and `pclass` columns simultaneously.

Finally, there are more advanced techniques that that imputation as a prediction task.
These techniques include MICE (multivariate imputation via chained equations) and other methods based on $k$-nearest neighbours or regression models.

In our class, we consider the more basic imputation techniques.

In [ ]:
# Impute "age" and "fare" using the group-by-group average computed
# over the groups defined by "pclass".
d <- d %>%
  group_by(pclass) %>%
  mutate(age = replace_na(age, mean(age, na.rm = TRUE)),
         fare = replace_na(fare, mean(fare, na.rm = TRUE))) %>%
  ungroup()

In [ ]:
# No NA's left.
summary(d)

    pclass    survived      name               sex           age       
 First :323   Yes:500   Length:1309        female:466   Min.   : 0.17  
 Second:277   No :809   Class :character   male  :843   1st Qu.:22.00  
 Third :709             Mode  :character                Median :26.00  
                                                        Mean   :29.35  
                                                        3rd Qu.:37.00  
                                                        Max.   :80.00  
 siblings_and_spouses_onboard parents_and_children_onboard    ticket         
 Min.   :0.0000               Min.   :0.000                Length:1309       
 1st Qu.:0.0000               1st Qu.:0.000                Class :character  
 Median :0.0000               Median :0.000                Mode  :character  
 Mean   :0.4989               Mean   :0.385                                  
 3rd Qu.:1.0000               3rd Qu.:0.000                                  
 Max.   :8.0000             

Our data no longer has missing values!

### Cannot impute using the label

We reinforce a basic concept of imputation: the **label** cannot be used for imputation; not even indirectly.

We have already stated that the label cannot be imputed, i.e., it cannot be used directly.

Moreover, the label cannot be used indirectly.
For example, we cannot impute different values depending on the label.
The reason is that, in production, the new data that comes in only has the features but not the label (because, indeed, we must *predict* the label).
Therefore, in production, it would be impossible to employ a technique that uses the label to impute feature values.

### Do not cause information leakage!

In the previous example, we have imputed the data directly into the `pandas` dataframe.
This means that we used potentially all observations to compute the statistics (mean in our case, median and mode in other cases) that were then used for imputation.

As you will soon see, when a dataset is used for a predictive task, it is often first divided into training data and test data.
The training data is used to train the machine learning models.
For parametric models, training involves learning good values for their parameters: values that enable the model to make accurate predictions when presented with new data.
During training, we select parameter values that yield a low prediction error, i.e., those that enable the model to predict labels that are close to the ground-truth label.

However, we are really interested in knowing how models will perform when deployed in production.
We also want to appraise their future performance before actually deploying them in production.
For example, if we must choose between two different models, we want to select the best one before it is deployed, not after.
To this end, we do not use all our available data for training; instead, we split the data into two parts.
The training data is used to train the models, while the test data is used as a "simulation" of future production data and is only used to evaluate the models.
For this evaluation to be unbiased, the test data must be kept completely separate from the training data.

Suppose we impute missing values in our dataset before this separation.
In that case, some test points will be included in the calculation of the statistics (such as the mean) used in data imputation.
Therefore, best practices involve using only training data for imputation and then applying the statistics computed on the training data to impute values in the test data.
This approach avoids *information leakage*, i.e., that some information contained in the test data leaks, however indirectly, into the training data.
In Python, this is achieved by using pipelines and including the imputation step in the pipeline.

You have not yet studied machine learning models and the `sklearn` library.
Come back to the following code in a couple of weeks to learn how to perform imputation correctly when the data you are imputing is used to train machine learning algorithms.

# An example of imputation + machine learning

In [ ]:
install.packages("rpart")
install.packages("caret")

library(caret)
library(rpart)
library(recipes)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Loading required package: ggplot2

Loading required package: lattice


Attaching package: ‘recipes’


The following object is masked from ‘package:stats’:

    step




In [ ]:
d = read_csv('/content/titanic.csv')
d = d %>%
  select(-c(name, ticket, body_id_number, cabin)) %>% # Remove columns that should not be used in a predictive task
  mutate(lifeboat = if_else(is.na(lifeboat), "No", "Yes")) %>% # Replace missing values with "No" and non-missing values with "Yes"
  mutate(pclass = as_factor(pclass),
         sex = as_factor(sex),
         embarkment_port = as_factor(embarkment_port),
         lifeboat = as_factor(lifeboat)) # Mark categorical columns as such

Rows: 1309 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): pclass, survived, name, sex, ticket, cabin, embarkment_port, lifeboat
dbl (5): age, siblings_and_spouses_onboard, parents_and_children_onboard, fa...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
train_index = createDataPartition(d$survived, p = 0.8, list = FALSE)
train_data = d[train_index, ]
test_data = d[-train_index, ]

In [ ]:
numerical_features = train_data %>%
  select(where(is.numeric)) %>%
  colnames()
categorical_features = train_data %>%
  select(where(is.factor)) %>%
  colnames() %>%
  setdiff("survived")

# Recipes are the closest thing to sklearn's pipelines
titanic_recipe = recipe(survived ~ ., data = train_data) %>%
  step_impute_mean(all_of(numerical_features)) %>%
  step_impute_mode(all_of(categorical_features)) %>%
  step_dummy(all_of(categorical_features))

# Still, the "fitting" of the preprocessing steps on the training data must be
# done manually, using the `prep` function.
prepared_recipe = prep(titanic_recipe, training = train_data)

# Once a recipe has been `prep`-ped on the training data, it can be `bake`-d
# on the test data. With sklearn's pipelines this was done automatically.
train_processed = bake(prepared_recipe, new_data = train_data)
test_processed = bake(prepared_recipe, new_data = test_data)

In [ ]:
param_grid = expand.grid(
  cp = c(0, 0.01, 0.1)
)

# 5-fold cross-validation
control = trainControl(method = "cv", number = 5)

tuned_rpart_model = train(
  survived ~ .,
  data = train_processed,
  method = "rpart",
  tuneGrid = param_grid,
  trControl = control
)

In [ ]:
# Homemade version of Python's GridSearchCV

# R's trees from package rpart do not have all the hyperparameters
# that Python's DecisionTreeClassifier has.
# We only use the "cp" pruning strength hyperparameter.
param_grid_cp = c(0, 0.0001, 0.001, 0.01, 0.01)

best_cp = 0
best_xerror = Inf

for(cp_value in param_grid_cp) {
  tuned_model = rpart(
    survived ~ .,
    data = train_processed,
    control = rpart.control(cp = cp_value),
    xval = 5)

  min_xerror_index = which.min(tuned_model$cptable[, "xerror"])
  current_xerror = tuned_model$cptable[min_xerror_index, "xerror"]

  if(current_xerror < best_xerror) {
    best_xerror = current_xerror
    best_cp = cp_value
  }
}

final_rpart_model = rpart(survived ~ ., data = train_processed, control = rpart.control(cp = best_cp))
print(paste("Best hyperparameter value found (cp):", best_cp))

[1] "Best hyperparameter value found (cp): 1e-04"


In [ ]:
predictions = predict(final_rpart_model, newdata = test_processed, type = "class")
accuracy = mean(predictions == test_processed$survived)
print(paste("Accuracy on the test set:", accuracy))

[1] "Accuracy on the test set: 0.980842911877395"


# Further references

* [Applied missing data analysis](https://www.appliedmissingdata.com/). Enders, Craig. Guilford (2022).
* [The missing book](https://tmb.njtierney.com/). Tierney, Nicholas and Horst, Allison. (2022).
* [Flexible imputation of missing data](https://stefvanbuuren.name/fimd/). van Buuren, Stef. CRC Press (2018).